<a href="https://colab.research.google.com/github/querzy/hello-world/blob/master/Model_2_LSTM_PREDICTIONS_MULTIPARITE_t0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

REINITIALISER L'ENVIRONNEMENT D'EXECUTION !!!
- Execution - reinitialiser l'environnement d'execution
- Executer le script plusieurs fois
- Utiliser le dossier Oanda currency download2 (4000 barres)
- Retirer une barre du fichier si barre ouverte et considérer le resultat à t0 OU considérer le résultat à t-1 avec une barre ouverte.

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
#@title
import numpy as np
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import DateOffset
from sklearn.preprocessing import MinMaxScaler
 
import tensorflow as tf
from tensorflow import keras
 
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import warnings
warnings.filterwarnings("ignore")
#import chart_studio.plotly as py
#import plotly.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
#pyoff.init_notebook_mode(connected=True)
from numpy.random import seed
seed(1)
tf.random.set_seed(2) 
 
 
#def parser(x):
#    return pd.datetime.strptime('190'+x, '%Y-%m')
 
 
#start='2020.08.21'
#rng=pd.date_range(start, periods=5, freq='B').strftime('%Y-%m-%d 21:00:00')
#df = pd.DataFrame({ 'date': rng, 'qty': np.random.randn(len(rng)) }) 
 
#plot daily prices
#plot_data = [
#    go.Scatter(
#        x=df['date'],
#        y=df['qty'],
#    )
#]
 
#plot_layout = go.Layout(
#        title='Daily Price'
#    )
#fig = go.Figure(data=plot_data, layout=plot_layout)
#fig.show()
 
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)
 
def get_num(xx):
    return int(''.join(ele for ele in xx if ele.isdigit()))


with open('/content/recap deep learning t0.txt', 'w') as document: pass

fileLst=['EURUSD1440.csv','EURUSD10080.csv','DOLLAR_INDX1440.csv','DOLLAR_INDX10080.csv','BTCUSD1440.csv','BTCUSD10080.csv'] 
  
for k in fileLst :
    filename1 = datetime.now().strftime("%d %b %Y - %H%M")
    filename=k.replace('.csv','')
    filename= filename+' '+filename1
    valperiod=get_num(k)
    print(valperiod)
    if valperiod==10080 :
        TimeFrame = 'Weekly'
        epochs=int(6)
    elif valperiod==1440 :
        TimeFrame = 'Daily'
        epochs=int(5)
    elif valperiod==240 :
        TimeFrame = '4H'
        epochs=int(3)
    elif valperiod==60 :
        TimeFrame = '1H'
        epochs=int(6)
    elif valperiod==15 :
        TimeFrame = '15 min'
        epochs=int(5)
 
    t0up=0
    t0dn=0 
    PRICEAVt0=[]
    PRICEAVupt0=[]
    PRICEAVdnt0=[]
    for j in range(20):
      df = pd.read_csv(k, index_col='Date',usecols=['Date', 'Close'],date_parser = pd.to_datetime)
      df.tail()
     
      print(df)
     
      train = df
     
      scaler = MinMaxScaler()
      scaler.fit(train)
      train = scaler.transform(train)
     
      n_input = 12
      n_features = 1
      generator = TimeseriesGenerator(train, train, length=n_input, batch_size=6)
     
      model = Sequential()
      #model.add(LSTM(200, activation='relu', input_shape=(n_input, n_features)))
      model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
      #model.add(Dropout(0.15))
      model.add(Dense(1))
     
      #Model summary
      #model.summary()
      #Compiling
      #model.compile(optimizer='adam', loss = 'mse')
     
      optimizer = keras.optimizers.Adam(learning_rate=0.001)
      model.compile(optimizer=optimizer, loss='mse')
     
      history = model.fit(generator,epochs=5,verbose=1)#EURUSD
      #history = model.fit(generator,epochs=20,verbose=1)#BTCUSD
     
      hist = pd.DataFrame(history.history)
      hist['epoch'] = history.epoch
     
      plot_data = [
          go.Scatter(
              x=hist['epoch'],
              y=hist['loss'],
              name='loss'
          )
          
      ]
     
      plot_layout = go.Layout(
              title='Training loss'
          )
      fig = go.Figure(data=plot_data, layout=plot_layout)
      #fig.show()
      pyoff.iplot(fig)
     
      pred_list = []
     
      batch = train[-n_input:].reshape((1, n_input, n_features))
     
      for i in range(n_input):   
          pred_list.append(model.predict(batch)[0]) 
          batch = np.append(batch[:,1:,:],[[pred_list[i]]],axis=1)
     
      if valperiod ==10080 :
        add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
      elif valperiod ==1440 :
        add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
      elif valperiod ==240 :
        add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]  
      elif valperiod ==60 :
        add_dates = [df.index[-1] + DateOffset(hours=x) for x in range(0,13) ] 
      #add_dates = [df.index[-1] + DateOffset(weeks=x) for x in range(0,13) ]
      #add_dates = [df.index[-1] + DateOffset(days=x) for x in range(0,13) ]
      #add_dates = [df.index[-1] + DateOffset(hours=x*4) for x in range(0,13) ]
      
      future_dates = pd.DataFrame(index=add_dates[1:],columns=df.columns)
      print(add_dates)
      df_predict = pd.DataFrame(scaler.inverse_transform(pred_list),
                                index=future_dates[-n_input:].index, columns=['Prediction'])
     
      df_proj = pd.concat([df,df_predict], axis=1)
     
      df_proj.tail(12)
     
      plot_data = [
          go.Scatter(
              x=df_proj.index[-30:],
              y=df_proj['Close'][-30:],
              name='actual'
          ),
          go.Scatter(
              x=df_proj.index[-30:],
              y=df_proj['Prediction'][-30:],
              name='prediction'
          )
      ]
     
      plot_layout = go.Layout(
              title=k+' prediction t0'
          )
     
      fig = go.Figure(data=plot_data, layout=plot_layout)
      #fig.show()
      pyoff.iplot(fig)
      
      
      yt0Cl=df['Close'].iloc[-1]
      yt0Pr=df_proj['Prediction'].iloc[-12]
      PRICEAVt0.append(yt0Pr)
      print(yt0Cl)
      print(yt0Pr)
      t0diff=yt0Cl-yt0Pr
      if t0diff>0:
        t0dn=t0dn+1
        PRICEAVdnt0.append(yt0Pr)
      elif t0diff<=0 :
        t0up=t0up+1
        PRICEAVupt0.append(yt0Pr)
          
    prixmoyt0=mean(PRICEAVt0)
    prixmoydnt0=mean(PRICEAVdnt0)
    prixmoyupt0=mean(PRICEAVupt0)

     
    
    print(' ')
    print('t0 :')
    print(' ')
    print('prix moyen t0 : ',prixmoyt0)
    print(' ')
    print('t0up : ',t0up)
    print('prix moyen up t0 : ',prixmoyupt0)
    print(' ')
    print('t0dn : ',t0dn)
    print('prix moyen dn t0 : ',prixmoydnt0)
    print(' ')
   

    with open('/content/recap deep learning t0.txt', 'a') as f:
        f.write(filename + '\n')
        f.write(' ' + '\n')
        f.write(' ' + '\n')
        f.write('t0 :'  + '\n')
        f.write(' '  + '\n')
        f.write('prix moyen t0 : '+str(prixmoyt0)  + '\n') 
        f.write(' '  + '\n')
        f.write('t0up : '+str(t0up)  + '\n')
        f.write('prix moyen up t0 : '+str(prixmoyupt0)  + '\n')
        f.write(' '  + '\n')
        f.write('t0dn : '+str(t0dn)  + '\n')
        f.write('prix moyen dn t0 : '+str(prixmoydnt0)  + '\n')
        f.write(' '  + '\n')
        f.write('####################'  + '\n')
        f.write(' '  + '\n')

f.close()    

from google.colab import files
files.download('/content/recap deep learning t0.txt')
